In [11]:
# Thư viện WebSocket để kết nối real-time với Bitget API
import websocket

# Thư viện xử lý JSON (dữ liệu từ API Bitget ở định dạng JSON)
import json

# Thư viện ghi file CSV để lưu dữ liệu ticker và candlestick
import csv

# Thư viện xử lý thời gian (timestamp, datetime conversion)
import datetime

# Thư viện xử lý file và thư mục (kiểm tra file tồn tại, kích thước file)
import os

# Thư viện pandas để xử lý dữ liệu dạng bảng (chưa sử dụng trong code này)
import pandas as pd

# Thư viện threading để chạy nhiều tác vụ cùng lúc (WebSocket chạy trong background)
import threading

# Thư viện time để tạm dừng chương trình và điều khiển thời gian chạy
import time

# Thư viện SQLAlchemy để kết nối database (đã import nhưng chưa sử dụng)
from sqlalchemy import create_engine

# Thư viện xử lý URL encoding (chưa sử dụng trong code này)
import urllib.parse

In [12]:
# ==================== CẤU HÌNH KẾT NỐI VÀ DỮ LIỆU ====================

# URL WebSocket public của Bitget để nhận dữ liệu real-time
WEBSOCKET_URL = "wss://ws.bitget.com/v2/ws/public"

# Danh sách các cặp cryptocurrency cần theo dõi (SOL, BTC, ETH với USDT)
INSTRUMENT_IDS = ["SOLUSDT", "BTCUSDT", "ETHUSDT"]  

# Tên file CSV để lưu dữ liệu ticker (giá hiện tại, volume, thay đổi 24h)
CSV_FILE_NAME = "data.csv"


# TRY CẬP VÀO products ĐỂ LẤY DANH SÁCH COIN

In [ ]:
import requests

def get_valid_trading_pairs():
    """Lấy danh sách trading pairs USDT hợp lệ từ Bitget API"""
    try:
        # Sử dụng API v2 để lấy symbols chính xác cho WebSocket
        url = "https://api.bitget.com/api/v2/spot/public/symbols"
        response = requests.get(url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            
            if data.get('code') == '00000' and 'data' in data:
                # Lọc chỉ các USDT pairs đang hoạt động
                valid_pairs = [
                    item.get('symbol') 
                    for item in data['data'] 
                    if (item.get('status') == 'online' and 
                        item.get('symbol', '').endswith('USDT'))
                ]
                
                print(f"✅ Lấy được {len(valid_pairs)} USDT trading pairs")
                return valid_pairs
            else:
                print(f"❌ API Error: {data.get('msg', 'Unknown error')}")
                return get_fallback_pairs()
        else:
            print(f"❌ HTTP Error: {response.status_code}")
            return get_fallback_pairs()
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Network Error: {e}")
        return get_fallback_pairs()
    except Exception as e:
        print(f"❌ Unexpected Error: {e}")
        return get_fallback_pairs()

def get_fallback_pairs():
    """Trả về danh sách trading pairs dự phòng khi API thất bại"""
    fallback_pairs = ["BTCUSDT", "ETHUSDT", "SOLUSDT", "BNBUSDT", "XRPUSDT"]
    print(f"🔄 Sử dụng {len(fallback_pairs)} trading pairs dự phòng")
    return fallback_pairs

def filter_top_pairs(pairs, limit=10):
    """Lọc top N trading pairs và loại bỏ các pairs có thể có vấn đề"""
    # Loại bỏ các pairs có tên lạ hoặc không phổ biến
    excluded_keywords = ['TEST', 'MOCK', 'DEMO', 'TEMP', '_SPBL']
    
    filtered_pairs = [
        pair for pair in pairs 
        if not any(keyword in pair.upper() for keyword in excluded_keywords)
    ]
    
    # Ưu tiên các pairs phổ biến lên đầu
    popular_pairs = ["BTCUSDT", "ETHUSDT", "SOLUSDT", "BNBUSDT", "XRPUSDT", 
                    "ADAUSDT", "DOTUSDT", "LINKUSDT", "UNIUSDT", "LTCUSDT"]
    priority_pairs = [pair for pair in popular_pairs if pair in filtered_pairs]
    other_pairs = [pair for pair in filtered_pairs if pair not in popular_pairs]
    
    # Kết hợp và lấy top N
    final_pairs = (priority_pairs + other_pairs)[:limit]
    
    return final_pairs

# ===== MAIN EXECUTION =====
print("🔍 Đang lấy danh sách trading pairs...")

# Lấy trading pairs từ API
all_trading_pairs = get_valid_trading_pairs()

# Lọc và lấy top 10 pairs an toàn
INSTRUMENT_IDS = filter_top_pairs(all_trading_pairs, limit=10)

# Verify kết quả cuối cùng
if not INSTRUMENT_IDS:
    print("⚠️ Không có trading pairs hợp lệ, sử dụng mặc định")
    INSTRUMENT_IDS = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]

print(f"🎯 Trading pairs được chọn: {INSTRUMENT_IDS}")
print(f"📊 Tổng số pairs: {len(INSTRUMENT_IDS)}")

792


# CẤU HÌNH HOÀN CHỈNH CHO TICKER + CANDLESTICK

In [ ]:


# URL WebSocket public của Bitget để nhận dữ liệu real-time
WEBSOCKET_URL = "wss://ws.bitget.com/v2/ws/public"

# Danh sách các cặp cryptocurrency cần theo dõi (đã được lọc)
# INSTRUMENT_IDS đã được set ở cell trước

# Tạo DataFrame để lưu dữ liệu ticker
ticker_df = pd.DataFrame(columns=[
    "thoi_gian", "timestamp_api", "instId", "lastPr", "bidPr", "askPr",
    "bidSz", "askSz", "open24h", "high24h", "low24h", "change24h",
    "baseVolume", "quoteVolume", "openUtc", "changeUtc24h", "action"
])

# Tạo DataFrame để lưu dữ liệu candlestick
candlestick_df = pd.DataFrame(columns=[
    "thoi_gian_he_thong", "start_time", "open_price", "high_price", 
    "low_price", "close_price", "volume_base", "volume_quote", 
    "volume_usdt", "instId", "timeframe", "action"
])

# Các khung thời gian candlestick cần theo dõi
TIMEFRAMES = ["candle1m", "candle5m", "candle15m", "candle1H"]

print("✅ DataFrames đã được khởi tạo")
print(f"📈 Ticker DataFrame: {len(ticker_df)} rows")
print(f"🕯️ Candlestick DataFrame: {len(candlestick_df)} rows")

# HÀM TẠO FILE CSV CHO DỮ LIỆU TICKER

In [15]:


def tao_file_csv():
    """Tạo file CSV với header chuẩn cho dữ liệu ticker"""
    
    # Định nghĩa các cột dữ liệu cho ticker (giá, volume, thống kê 24h)
    header = [
    "thoi_gian" ,          # Thời gian hệ thống ghi nhận dữ liệu (local time)
    "timestamp_api",       # Timestamp từ API Bitget (ms)
    "instId",              # Mã sản phẩm (ví dụ: ETHUSDT, BTCUSDT)
    "lastPr",              # Giá giao dịch gần nhất (current price)
    "bidPr",               # Giá mua cao nhất (highest bid)
    "askPr",               # Giá bán thấp nhất (lowest ask)
    "bidSz",               # Khối lượng đặt mua tại giá bid
    "askSz",               # Khối lượng đặt bán tại giá ask
    "open24h",             # Giá mở cửa 24 giờ trước
    "high24h",             # Giá cao nhất trong 24h
    "low24h",              # Giá thấp nhất trong 24h
    "change24h",           # Phần trăm thay đổi trong 24h
    "baseVolume",          # Khối lượng giao dịch theo base currency (coin)
    "quoteVolume",         # Khối lượng giao dịch theo quote currency (USDT)
    "openUtc",             # Giá mở cửa tại UTC+0
    "changeUtc24h",        # Thay đổi giá tại UTC+0 trong 24h
    "action"               # Loại dữ liệu: snapshot (cũ) hoặc update (mới)
]

    # Kiểm tra file có tồn tại và có dữ liệu chưa
    if not os.path.exists(CSV_FILE_NAME) or os.path.getsize(CSV_FILE_NAME) == 0:
        # Tạo file mới với header nếu chưa tồn tại
        with open(CSV_FILE_NAME, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(header)
        print(f"Đã tạo file CSV với {len(header)} cột: {CSV_FILE_NAME}")
    else:
        print(f"File CSV đã tồn tại: {CSV_FILE_NAME}")

# Gọi hàm tạo file ticker ngay khi chạy
tao_file_csv()

File CSV đã tồn tại: data.csv


# HÀM TẠO FILE CSV CHO DỮ LIỆU CANDLESTICK

In [16]:


def tao_file_candlestick_csv():
    """Tạo file CSV với header chuẩn cho dữ liệu candlestick (nến OHLC)"""
    
    # Định nghĩa các cột dữ liệu cho candlestick (nến OHLC + volume)
    candlestick_header = [
        "thoi_gian_he_thong",      # Thời gian hệ thống ghi nhận (local time)
    "start_time",              # Thời gian bắt đầu nến (từ API, đã convert sang readable)
    "open_price",              # Giá mở cửa của nến (Open)
    "high_price",              # Giá cao nhất của nến (High)
    "low_price",               # Giá thấp nhất của nến (Low)
    "close_price",             # Giá đóng cửa của nến (Close)
    "volume_base",             # Khối lượng giao dịch theo base currency
    "volume_quote",            # Khối lượng giao dịch theo quote currency
    "volume_usdt",             # Khối lượng giao dịch tính bằng USDT
    "instId",                  # Mã sản phẩm (ETHUSDT, BTCUSDT, SOLUSDT)
    "timeframe",               # Khung thời gian nến (1m, 5m, 15m, 1H)
    "action"                   # Loại dữ liệu: snapshot hoặc update
]
    
    # Kiểm tra file candlestick có tồn tại chưa
    if not os.path.exists(CANDLESTICK_CSV_FILE) or os.path.getsize(CANDLESTICK_CSV_FILE) == 0:
        # Tạo file mới với header nếu chưa tồn tại
        with open(CANDLESTICK_CSV_FILE, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(candlestick_header)
        print(f"Đã tạo file Candlestick CSV với {len(candlestick_header)} cột: {CANDLESTICK_CSV_FILE}")
    else:
        print(f"File Candlestick CSV đã tồn tại: {CANDLESTICK_CSV_FILE}")

# Gọi hàm tạo file candlestick ngay khi chạy
tao_file_candlestick_csv()

File Candlestick CSV đã tồn tại: candlestick_data.csv


# BIẾN TOÀN CỤC VÀ HÀM XỬ LÝ WEBSOCKET

In [ ]:


# Mảng lưu trữ toàn bộ dữ liệu ticker và candlestick để debug
all_ticker_data = []
all_candlestick_data = []

def on_open_enhanced(ws):
    """Hàm được gọi khi WebSocket kết nối thành công"""
    print("🔗 Đã kết nối thành công với Bitget WebSocket")
    
    # === ĐĂNG KÝ THEO DÕI TICKER CHO TẤT CẢ SYMBOLS ===
    for inst_id in INSTRUMENT_IDS:
        ticker_message = {
            "op": "subscribe",        
            "args": [
                {    
                    "instType": "SPOT",      
                    "channel": "ticker",     
                    "instId": inst_id        
                }
            ]
        }
        ws.send(json.dumps(ticker_message))
        print(f"📊 Đăng ký ticker: {inst_id}")
    
    # === ĐĂNG KÝ THEO DÕI CANDLESTICK CHO TẤT CẢ SYMBOLS VÀ TIMEFRAMES ===
    for inst_id in INSTRUMENT_IDS:
        for timeframe in TIMEFRAMES:
            candlestick_message = {
                "op": "subscribe",
                "args": [
                    {    
                        "instType": "SPOT",      
                        "channel": timeframe,    
                        "instId": inst_id        
                    }
                ]
            }
            ws.send(json.dumps(candlestick_message))
            print(f"🕯️ Đăng ký candlestick: {inst_id} - {timeframe}")
    
    print(f"✅ Hoàn tất subscribe cho {len(INSTRUMENT_IDS)} coins với {len(TIMEFRAMES)} timeframes")

def on_message_enhanced(ws, message_str):
    """Hàm xử lý tin nhắn từ WebSocket - route dữ liệu đến đúng processor"""
    global all_ticker_data, all_candlestick_data
    
    # Parse JSON từ message
    data = json.loads(message_str)
    
    # Kiểm tra message type
    if "event" in data:
        if data["event"] == "error":
            print(f"❌ WebSocket Error: {data.get('msg', 'Unknown error')}")
            return
        elif data["event"] == "subscribe":
            print(f"✅ Subscribed: {data.get('arg', {}).get('instId', 'Unknown')} - {data.get('arg', {}).get('channel', 'Unknown')}")
            return
    
    # Kiểm tra loại channel để route dữ liệu đúng chỗ
    if "arg" in data and "channel" in data["arg"]:
        channel = data["arg"]["channel"]
        
        if channel == "ticker":
            # Dữ liệu ticker: giá hiện tại, volume, bid/ask
            all_ticker_data.append(data)
            process_ticker_data(data)
            
        elif channel.startswith("candle"):
            # Dữ liệu candlestick: nến OHLC với các timeframe khác nhau
            all_candlestick_data.append(data)
            process_candlestick_data(data)

def process_ticker_data(data):
    """Xử lý và lưu dữ liệu ticker vào DataFrame"""
    global ticker_df
    
    if "data" in data and data["data"]:
        ticker = data["data"][0]  
        
        # === TRÍCH XUẤT DỮ LIỆU TICKER ===
        new_row = {
            "thoi_gian": datetime.datetime.now().isoformat(),
            "timestamp_api": ticker.get('ts'),
            "instId": ticker.get('instId'),
            "lastPr": ticker.get('lastPr'),
            "bidPr": ticker.get('bidPr'),
            "askPr": ticker.get('askPr'),
            "bidSz": ticker.get('bidSz'),
            "askSz": ticker.get('askSz'),
            "open24h": ticker.get('open24h'),
            "high24h": ticker.get('high24h'),
            "low24h": ticker.get('low24h'),
            "change24h": ticker.get('change24h'),
            "baseVolume": ticker.get('baseVolume'),
            "quoteVolume": ticker.get('quoteVolume'),
            "openUtc": ticker.get('openUtc'),
            "changeUtc24h": ticker.get('changeUtc24h'),
            "action": data.get('action', 'unknown')
        }
        
        # === THÊM DỮ LIỆU VÀO DATAFRAME ===
        ticker_df = pd.concat([ticker_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # === HIỂN THỊ THÔNG TIN TICKER TRÊN CONSOLE ===
        print(f"📈 TICKER {new_row['instId']} | Giá: {new_row['lastPr']} | Thay đổi 24h: {new_row['change24h']} | Total: {len(ticker_df)}")

def process_candlestick_data(data):
    """Xử lý và lưu dữ liệu candlestick (nến OHLC) vào DataFrame"""
    global candlestick_df
    
    if "data" in data and data["data"]:
        candle = data["data"][0]  
        arg = data.get("arg", {})
        
        # === CONVERT TIMESTAMP SANG ĐỊNH DẠNG DỄ ĐỌC ===
        try:
            start_time_readable = datetime.datetime.fromtimestamp(int(candle[0])/1000).isoformat()
        except:
            start_time_readable = candle[0]
        
        # === TRÍCH XUẤT DỮ LIỆU TỪ CANDLE ARRAY ===
        new_row = {
            "thoi_gian_he_thong": datetime.datetime.now().isoformat(),
            "start_time": start_time_readable,
            "open_price": candle[1],
            "high_price": candle[2],
            "low_price": candle[3],
            "close_price": candle[4],
            "volume_base": candle[5],
            "volume_quote": candle[6],
            "volume_usdt": candle[7],
            "instId": arg.get('instId'),
            "timeframe": arg.get('channel'),
            "action": data.get('action', 'unknown')
        }
        
        # === THÊM DỮ LIỆU VÀO DATAFRAME ===
        candlestick_df = pd.concat([candlestick_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # === HIỂN THỊ THÔNG TIN CANDLESTICK TRÊN CONSOLE ===
        print(f"🕯️ CANDLE {new_row['instId']} {new_row['timeframe']} | O:{new_row['open_price']} H:{new_row['high_price']} L:{new_row['low_price']} C:{new_row['close_price']} | Total: {len(candlestick_df)}")

def on_error(ws, error):
    """Hàm xử lý lỗi WebSocket"""
    print(f"❌ WebSocket Error: {error}")

def on_close(ws, close_status_code, close_msg):
    """Hàm xử lý khi WebSocket đóng kết nối"""
    print(f"🔌 Kết nối WebSocket đã đóng")

# CHẠY WEBSOCKET VÀ ĐIỀU KHIỂN CHƯƠNG TRÌNH

In [ ]:
# === CẤU HÌNH THỜI GIAN CHẠY ===
a = 60  # 1 phút = 60 giây
b = a * 60  # 1 giờ = 3600 giây
c = b * 24  # 1 ngày = 86400 giây

def run_ws_enhanced():
    """Hàm chạy WebSocket với ping để duy trì kết nối"""
    ws.run_forever(ping_interval=30, ping_timeout=10)

# === TẠO ĐỐI TƯỢNG WEBSOCKET VỚI CÁC CALLBACK FUNCTIONS ===
ws = websocket.WebSocketApp(WEBSOCKET_URL,
                          on_open=on_open_enhanced,      
                          on_message=on_message_enhanced,  
                          on_error=on_error,            
                          on_close=on_close)            

# === THÔNG BÁO BẮT ĐẦU CRAWLING ===
print("🚀 Bắt đầu kết nối đến Bitget WebSocket...")
print("💾 Dữ liệu sẽ được lưu vào DataFrames")
print("⏹️ Nhấn Ctrl+C để dừng")

# === CHẠY WEBSOCKET TRONG BACKGROUND THREAD ===
ws_thread = threading.Thread(target=run_ws_enhanced)
ws_thread.daemon = True  
ws_thread.start()

# === THIẾT LẬP THỜI GIAN CHẠY ===
run_duration = a # Chạy trong 1 phút (60 giây)

# === CHẠY CHƯƠNG TRÌNH VÀ XỬ LÝ KEYBOARD INTERRUPT ===
try:
    time.sleep(run_duration)  
except KeyboardInterrupt:
    print("\n⏹️ Đã dừng bằng Ctrl+C")  

# === ĐÓNG KẾT NỐI VÀ THỐNG KÊ ===
ws.close()
print(f"✅ Đã ngắt kết nối sau {run_duration} giây.")
print(f"📊 Ticker data: {len(all_ticker_data)} messages")        
print(f"🕯️ Candlestick data: {len(all_candlestick_data)} messages")
print(f"📈 Ticker DataFrame: {len(ticker_df)} rows")
print(f"🕯️ Candlestick DataFrame: {len(candlestick_df)} rows")

Bắt đầu kết nối đến Bitget (Ticker + Candlestick)...
Ticker data sẽ được lưu vào: data.csv
Candlestick data sẽ được lưu vào: candlestick_data.csv
Nhấn Ctrl+C để dừng
Đã kết nối thành công
Đang theo dõi ticker LUMIAUSDT_SPBL
Đang theo dõi ticker GOATUSDT_SPBL
Đang theo dõi ticker ENAEUR_SPBL
Đang theo dõi ticker TONCOINEUR_SPBL
Đang theo dõi ticker SXRPSUSDT_SPBL
Đang theo dõi ticker SHIBEUR_SPBL
Đang theo dõi ticker WLDEUR_SPBL
Đang theo dõi ticker DBRUSDT_SPBL
Đang theo dõi ticker ACTUSDT_SPBL
Đang theo dõi ticker GRASSUSDT_SPBL
Đang theo dõi ticker NEIROCTOEUR_SPBL
Đang theo dõi ticker SUIEUR_SPBL
Đang theo dõi ticker TRXUSDT_SPBL
Đang theo dõi ticker XNEWUSDT_SPBL
Đang theo dõi ticker LINKUSDT_SPBL
Đang theo dõi ticker ONDOEUR_SPBL
Đang theo dõi ticker UNIUSDT_SPBL
Đang theo dõi ticker SUSHIUSDT_SPBL
Đang theo dõi ticker COMPUSDT_SPBL
Đang theo dõi ticker SCREUR_SPBL
Đang theo dõi ticker AAVEUSDT_SPBL
Đang theo dõi ticker SCRUSDT_SPBL
Đang theo dõi ticker YFIUSDT_SPBL
Đang theo dõi 

In [19]:
# Hiển thị raw data từ ticker messages
print("=== RAW TICKER DATA ===")
print(f"Total ticker messages: {len(all_ticker_data)}")
print("\nFirst 3 ticker messages:")
for i, msg in enumerate(all_ticker_data[:3]):
    print(f"\nMessage {i+1}:")
    print(json.dumps(msg, indent=2))
    
print("\nLast 3 ticker messages:")
for i, msg in enumerate(all_ticker_data[-3:], len(all_ticker_data)-2):
    print(f"\nMessage {i}:")
    print(json.dumps(msg, indent=2))

=== RAW TICKER DATA ===
Total ticker messages: 49

First 3 ticker messages:

Message 1:
{
  "event": "error",
  "arg": {
    "instType": "SPOT",
    "channel": "ticker",
    "instId": "LUMIAUSDT_SPBL"
  },
  "code": 30001,
  "msg": "instType:SPOT,channel:ticker,instId:LUMIAUSDT_SPBL doesn't exist",
  "op": "subscribe"
}

Message 2:
{
  "event": "error",
  "arg": {
    "instType": "SPOT",
    "channel": "ticker",
    "instId": "GOATUSDT_SPBL"
  },
  "code": 30001,
  "msg": "instType:SPOT,channel:ticker,instId:GOATUSDT_SPBL doesn't exist",
  "op": "subscribe"
}

Message 3:
{
  "event": "error",
  "arg": {
    "instType": "SPOT",
    "channel": "ticker",
    "instId": "ENAEUR_SPBL"
  },
  "code": 30001,
  "msg": "instType:SPOT,channel:ticker,instId:ENAEUR_SPBL doesn't exist",
  "op": "subscribe"
}

Last 3 ticker messages:

Message 47:
{
  "event": "error",
  "arg": {
    "instType": "SPOT",
    "channel": "ticker",
    "instId": "OORTUSDT_SPBL"
  },
  "code": 30001,
  "msg": "instType:SPO

In [20]:
# Hiển thị raw data từ candlestick messages
print("=== RAW CANDLESTICK DATA ===")
print(f"Total candlestick messages: {len(all_candlestick_data)}")
print("\nFirst 3 candlestick messages:")
for i, msg in enumerate(all_candlestick_data[:3]):
    print(f"\nMessage {i+1}:")
    print(json.dumps(msg, indent=2))
    
print("\nLast 3 candlestick messages:")
for i, msg in enumerate(all_candlestick_data[-3:], len(all_candlestick_data)-2):
    print(f"\nMessage {i}:")
    print(json.dumps(msg, indent=2))

=== RAW CANDLESTICK DATA ===
Total candlestick messages: 0

First 3 candlestick messages:

Last 3 candlestick messages:


In [21]:
# Phân tích các loại message trong raw data
print("=== PHÂN TÍCH RAW DATA ===")

# Phân tích ticker messages
ticker_event_types = {}
ticker_action_types = {}

for msg in all_ticker_data:
    # Đếm event types
    event = msg.get('event', 'no_event')
    ticker_event_types[event] = ticker_event_types.get(event, 0) + 1
    
    # Đếm action types
    action = msg.get('action', 'no_action')
    ticker_action_types[action] = ticker_action_types.get(action, 0) + 1

print("TICKER MESSAGE TYPES:")
print(f"Event types: {ticker_event_types}")
print(f"Action types: {ticker_action_types}")

# Phân tích candlestick messages
candle_event_types = {}
candle_action_types = {}
candle_timeframes = {}

for msg in all_candlestick_data:
    # Đếm event types
    event = msg.get('event', 'no_event')
    candle_event_types[event] = candle_event_types.get(event, 0) + 1
    
    # Đếm action types
    action = msg.get('action', 'no_action')
    candle_action_types[action] = candle_action_types.get(action, 0) + 1
    
    # Đếm timeframes
    if 'arg' in msg and 'channel' in msg['arg']:
        timeframe = msg['arg']['channel']
        candle_timeframes[timeframe] = candle_timeframes.get(timeframe, 0) + 1

print("\nCANDLESTICK MESSAGE TYPES:")
print(f"Event types: {candle_event_types}")
print(f"Action types: {candle_action_types}")
print(f"Timeframes: {candle_timeframes}")

=== PHÂN TÍCH RAW DATA ===
TICKER MESSAGE TYPES:
Event types: {'error': 49}
Action types: {'no_action': 49}

CANDLESTICK MESSAGE TYPES:
Event types: {}
Action types: {}
Timeframes: {}


In [22]:
# Lưu raw data vào file JSON để backup
import json

# Lưu ticker raw data
with open('raw_ticker_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_ticker_data, f, indent=2, ensure_ascii=False)

# Lưu candlestick raw data  
with open('raw_candlestick_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_candlestick_data, f, indent=2, ensure_ascii=False)

print("Raw data đã được lưu vào:")
print("- raw_ticker_data.json")
print("- raw_candlestick_data.json")
print(f"Ticker messages: {len(all_ticker_data)}")
print(f"Candlestick messages: {len(all_candlestick_data)}")

Raw data đã được lưu vào:
- raw_ticker_data.json
- raw_candlestick_data.json
Ticker messages: 49
Candlestick messages: 0


In [23]:
# Hiển thị sample data structure cho mỗi loại message
print("=== SAMPLE DATA STRUCTURES ===")

# Tìm một ticker message có data
ticker_with_data = None
for msg in all_ticker_data:
    if 'data' in msg and msg['data']:
        ticker_with_data = msg
        break

if ticker_with_data:
    print("TICKER MESSAGE WITH DATA:")
    print(json.dumps(ticker_with_data, indent=2))
    print(f"\nTicker data fields: {list(ticker_with_data['data'][0].keys())}")

# Tìm một candlestick message có data
candle_with_data = None
for msg in all_candlestick_data:
    if 'data' in msg and msg['data']:
        candle_with_data = msg
        break

if candle_with_data:
    print("\n" + "="*50)
    print("CANDLESTICK MESSAGE WITH DATA:")
    print(json.dumps(candle_with_data, indent=2))
    print(f"\nCandlestick data array length: {len(candle_with_data['data'][0])}")
    print("Candlestick data format: [timestamp, open, high, low, close, base_volume, quote_volume, usdt_volume]")

=== SAMPLE DATA STRUCTURES ===
